# Write data to .mseed files

### Imports & Methods

In [1]:
import sys
import obspy as obs

from andbro__querrySeismoData import __querrySeismoData

In [2]:
def __user_interaction():

    conf = {}

    conf['seed'] = input("\nEnter seed name:  ") or None

    if conf['seed'] is None:
        print(" -> No seed id provided!")
        sys.exit()

    conf['repository'] = input("\nEnter repository (archive / local / [george]):  ") or "george"  
    
    if conf['repository'].lower() == 'local':
        conf['datapath'] = input("\nEnter datapath:  ")
    else:
        conf['datapath'] = None


    ## ask for time period
    conf['tbeg'], conf['tend'] = None, None
    while conf['tbeg'] is None:
        conf['tbeg']  = obs.UTCDateTime(input("\nEnter start time (e.g. 2020-06-29 09:52):  ")) or None

    while conf['tend'] is None:
        conf['tend']  = obs.UTCDateTime(input("\nEnter end time (e.g. 2020-06-29 10:00):  ")) or None

    conf['outpath'] = input("\nEnter output path:  ") or None
    
    if conf['outpath'] is None:
        print(" -> No output path id provided!")
        sys.exit()
    if conf['outpath'][-1] != "/":
        conf['outpath'] += "/"
        
    conf['outformat'] = input("\nEnter output file format (mseed | ascii):  ") or None

    if conf['outformat'] == "ascii":
        conf['outformat_type'] = "SLIST"
    else:
        conf['outformat_type'] = conf['outformat']
        
        
        
#     ## ask for filter parameters
#     conf['set_filter'] = input("\nSet Filter (yes/no)?  ") or None

#     if conf['set_filter'].lower() in ["yes", "y"]:
#         conf['filter_type'] = input("\nEnter filter type (bp, lp, hp): ")

#         if conf['filter_type'].lower() in ['bp', 'bandpass']:
#             conf['filter_type'] = 'bandpass'
#             conf['lower_corner_frequency'] = float(input("\nEnter lower corner frequency (in Hz): ")) or None
#             conf['upper_corner_frequency'] = float(input("Enter upper corner frequency (in Hz): ")) or None

#         elif conf['filter_type'].lower() in ['hp', 'highpass']:
#             conf['filter_type'] = 'highpass'
#             conf['lower_corner_frequency'] = float(input("\nEnter lower corner frequency (in Hz): ")) or None
#             conf['upper_corner_frequency'] = None

#         elif conf['filter_type'].lower() in ['lp', 'lowpass']:
#             conf['filter_type'] = 'lowpass'
#             conf['lower_corner_frequency'] = None
#             conf['upper_corner_frequency'] = float(input("\nEnter upper corner frequency (in Hz): ")) or None

    return conf

### Configurations

In [3]:
config = __user_interaction()


Enter seed name:  BW.DROMY..F2V

Enter repository (archive / local / [george]):  archive

Enter start time (e.g. 2020-06-29 09:52):  2022-03-31 19:00

Enter end time (e.g. 2020-06-29 10:00):  2022-03-31 21:00

Enter output path:  /home/brotzer/Downloads/

Enter output file format (mseed | ascii):  mseed


### Load Data from Repository

In [4]:
st0, inv = __querrySeismoData(
                            seed_id=config.get("seed"),
                            starttime=config.get("tbeg"),
                            endtime=config.get("tend"),
                            repository=config.get("repository"),
                            path=config['datapath'],
                            restitute=False,
                            detail=True,
                            fill_value=None,
                            )
st0

Coordinates could not be attached to Inventory!


1 Trace(s) in Stream:
BW.DROMY..F2V | 2022-03-31T19:00:00.000000Z - 2022-03-31T21:00:00.000000Z | 5000.0 Hz, 36000001 samples

### Write Data to Output

In [5]:
for tr in st0:
    sta = config['seed'].split(".")[1]
    cha = config['seed'].split(".")[3]
    tbeg_date = config['tbeg'].date

    config['outname'] = f"{sta}_{cha}_{tbeg_date}.{config['outformat']}"

    tr.write(config['outpath']+config['outname'], format=config['outformat_type'])